---------------------
````sql
CREATE TABLE sales_data (
    transaction_id INT,
    product_id INT,
    customer_id INT,
    transaction_date DATE,
    quantity INT,
    price_per_unit INT
);
````
------------------
````sql

INSERT INTO sales_data 
(transaction_id, product_id, customer_id, transaction_date, quantity, price_per_unit)
VALUES
-- 2021
(1, 101, 1, '2021-01-10', 2, 500),
(2, 101, 2, '2021-03-15', 1, 500),
(3, 102, 1, '2021-08-20', 3, 300),

-- 2022
(4, 101, 1, '2022-02-12', 2, 550),
(5, 102, 3, '2022-04-22', 4, 320),
(6, 103, 4, '2022-09-18', 1, 800),

-- 2023
(7, 101, 2, '2023-01-05', 3, 600),
(8, 102, 1, '2023-06-14', 2, 350),
(9, 103, 4, '2023-11-11', 2, 900);
````
-----------------------------------------------------------------
````sql
-- 1. Calculate the total revenue for each product.
SELECT product_id, sum(quantity*price_per_unit)
From Sales_data
Group BY Product_id;

-- 2. Calculate YoY (Year-on-Year) revenue growth for each product.
WITH CTE1 AS (
    SELECT 
        product_id, 
        YEAR(transaction_date) AS year,
        SUM(quantity * price_per_unit) AS revenue
    FROM sales_data
    GROUP BY product_id, YEAR(transaction_date)
),
CTE2 AS (
    SELECT 
        product_id,
        year,
        revenue,
        LAG(revenue, 1) OVER (PARTITION BY product_id ORDER BY year) AS previous_year_revenue
    FROM CTE1
)
SELECT
    product_id,
    year,
    revenue,
    previous_year_revenue,
    ROUND(((revenue - previous_year_revenue) / previous_year_revenue) * 100, 2)
    AS yoy_growth_percent
FROM CTE2
ORDER BY product_id, year;

SELECT
    product_id,
    year,
    revenue,
    previous_year_revenue,
    CASE 
        WHEN previous_year_revenue IS NULL THEN NULL
        ELSE ROUND(((revenue - previous_year_revenue) / previous_year_revenue) * 100, 2)
    END AS yoy_growth_percent
FROM CTE2
ORDER BY product_id, year;

-- 3. Find the year-wise revenue for each product.
SELECT Product_id, Year(Transaction_date) AS Year, SUM(Quantity*Price_Per_Unit) AS Revenue
FROM Sales_Data
Group BY Product_id, Year(Transaction_date);

-- 4. Find the first transaction date and last transaction date for each customer.
SELECT Customer_id, MIN(Transaction_Date) AS First_transaction_date, MAX(Transaction_Date) AS LAST_transaction_date
From Sales_data
Group by Customer_id;

-- 5. Find the top 2 highest-spending customers overall.
WITH CTE1 AS(Select Customer_id, SUM(Quantity * Price_Per_Unit) AS Spending
From Sales_Data
Group By Customer_id),

CTE2 AS (SELECT customer_id, Spending, 
dense_rank() OVER(Order BY Spending DESC) AS Ranking FROM CTE1)

SELECT Customer_id, Spending From CTE2 Where Ranking <= 2;

-- 6. For each customer, find their most frequently purchased product.
WITH CTE1 AS (
    SELECT 
        customer_id,
        product_id,
        COUNT(*) AS purchase_count
    FROM sales_data
    GROUP BY customer_id, product_id
),
CTE2 AS (
    SELECT
        customer_id,
        product_id,
        purchase_count,
        DENSE_RANK() OVER (
            PARTITION BY customer_id 
            ORDER BY purchase_count DESC
        ) AS ranking
    FROM CTE1
)
SELECT
    customer_id,
    product_id,
    purchase_count
FROM CTE2
WHERE ranking = 1;

-- 7. Calculate running total revenue for each product sorted by date.
SELECT
    product_id,
    transaction_date,
    quantity * price_per_unit AS revenue,
    SUM(quantity * price_per_unit) 
        OVER (
            PARTITION BY product_id 
            ORDER BY transaction_date
        ) AS running_total_revenue
FROM sales_data
ORDER BY product_id, transaction_date;


-- 8. Find customers who bought from more than one product category.
WITH CTE1 AS(Select Customer_id, COUNT(Distinct Product_id) AS Product_Count
FROM Sales_data
Group BY Customer_id)

Select * FROM CTE1 Where Product_Count > 1;

-- 9. Identify months with the highest total revenue.
WITH CTE1 AS (
    SELECT 
        YEAR(transaction_date) AS year,
        MONTH(transaction_date) AS month,
        SUM(quantity * price_per_unit) AS total_revenue
    FROM sales_data
    GROUP BY 
        YEAR(transaction_date), 
        MONTH(transaction_date)
),
CTE2 AS (
    SELECT
        year,
        month,
        total_revenue,
        DENSE_RANK() OVER (ORDER BY total_revenue DESC) AS ranking
    FROM CTE1
)
SELECT 
    year,
    month,
    total_revenue
FROM CTE2
WHERE ranking = 1;

-- 10. Find transactions where quantity purchased was above the average quantity for that product.
WITH CTE1 AS(Select *, AVG(Quantity) OVER(Partition BY Product_id) AS Average_Quantity
FROM Sales_Data)
Select transaction_id, Product_id, Customer_id, Transaction_date, quantity, Price_Per_unit
FROM CTE1 Where Quantity > Average_Quantity;